## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Explain Clinical Document - Cancer Type


- **Model**: `explain_clinical_doc_cancer_type_en`
- **Model Description**: This pipeline extracts oncological and cancer type entities, assign assertion status to the extracted entities and establish relations between the extracted entities from the clinical documents.


In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/base_serializers.py:28: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.22.4)
  import scipy.sparse


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

In [4]:
model_name = "explain-clinical-doc-cancer-type-en"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.2xlarge"

## 2. Create a deployable model from the model package.

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session,
)

### Input Format

To use the model, you need to provide input in one of the following supported formats:

#### JSON Format

Provide input as JSON. We support two variations within this format:

1. **Array of Text Documents**: 
   Use an array containing multiple text documents. Each element represents a separate text document.

   ```json
   {
       "text": [
           "Text document 1",
           "Text document 2",
           ...
       ]
   }

    ```

2. **Single Text Document**:
   Provide a single text document as a string.


   ```json
    {
        "text": "Single text document"
    }
   ```

#### JSON Lines (JSONL) Format

Provide input in JSON Lines format, where each line is a JSON object representing a text document.

```
{"text": "Text document 1"}
{"text": "Text document 2"}
```

## 3. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Deploy the SageMaker model to an endpoint

In [6]:
# Deploy the model
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
)

----------!

Once endpoint has been created, you would be able to perform real-time inference.

In [7]:
import json
import pandas as pd
import os
import boto3

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

def process_data_and_invoke_realtime_endpoint(data, content_type, accept):

    content_type_to_format = {'application/json': 'json', 'application/jsonlines': 'jsonl'}
    input_format = content_type_to_format.get(content_type)
    if content_type not in content_type_to_format.keys() or accept not in content_type_to_format.keys():
        raise ValueError("Invalid content_type or accept. It should be either 'application/json' or 'application/jsonlines'.")

    i = 1
    input_dir = f'inputs/real-time/{input_format}'
    output_dir = f'outputs/real-time/{input_format}'
    s3_input_dir = f"{model_name}/validation-input/real-time/{input_format}"
    s3_output_dir = f"{model_name}/validation-output/real-time/{input_format}"

    input_file_name = f'{input_dir}/input{i}.{input_format}'
    output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    while os.path.exists(input_file_name) or os.path.exists(output_file_name):
        i += 1
        input_file_name = f'{input_dir}/input{i}.{input_format}'
        output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
    os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

    input_data = json.dumps(data) if content_type == 'application/json' else data

    # Write input data to file
    with open(input_file_name, 'w') as f:
        f.write(input_data)

    # Upload input data to S3
    s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_input_dir}/{os.path.basename(input_file_name)}", Body=bytes(input_data.encode('UTF-8')))

    # Invoke the SageMaker endpoint
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=input_data,
    )

    # Read response data
    response_data = json.loads(response["Body"].read().decode("utf-8")) if accept == 'application/json' else response['Body'].read().decode('utf-8')

    # Save response data to file
    with open(output_file_name, 'w') as f_out:
        if accept == 'application/json':
            json.dump(response_data, f_out, indent=4)
        else:
            for item in response_data.split('\n'):
                f_out.write(item + '\n')

    # Upload response data to S3
    output_s3_key = f"{s3_output_dir}/{os.path.basename(output_file_name)}"
    if accept == 'application/json':
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=json.dumps(response_data).encode('UTF-8'))
    else:
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=response_data)

    return response_data

### Initial Setup

In [8]:
docs = [
        """She underwent a computed tomography (CT) scan of the abdomen and pelvis, which showed a complex ovarian mass. A Pap smear performed one month later was positive for atypical glandular cells suspicious for adenocarcinoma. The pathologic specimen showed extension of the tumor throughout the fallopian tubes, appendix, omentum, and 5 out of 5 enlarged lymph nodes. The final pathologic diagnosis of the tumor was stage IIIC papillary serous ovarian adenocarcinoma. Two months later, the patient was diagnosed with lung metastases.""",

        """Chest computed tomography (CT) showed pulmonary lesions in posterior segment of right upper lobe, and peripheral lung cancer with multiple pulmonary metastases. Multiple metastases of the thoracic vertebrae, sternum, and ribs were considered, which were similar to previous CT images."""
]

sample_text = """A 56-year-old man presented with a 2-month history of whole-body weakness, double vision, difficulty swallowing, and a 45 mm anterior mediastinal mass detected via chest CT. Neurological examination and electromyography confirmed a diagnosis of Lambert-Eaton Myasthenic Syndrome (LEMS), associated with anti-P/Q-type VGCC antibodies. The patient was treated with
cisplatin 75 mg/m² on day 1, combined with etoposide 100 mg/m² on days 1-3, repeated every 3 weeks for four cycles. A video-assisted thoracic surgery revealed histopathological features consistent
with small cell lung cancer (SCLC) with lymph node metastases. The immunohistochemical analysis showed positive markers for AE1/AE3, TTF-1, chromogranin A, and synaptophysin. Notably,
a pulmonary nodule in the left upper lobe disappeared, and FDG-PET/CT post-surgery revealed no primary lesions or metastases."""

### JSON

#### Example 1

In [9]:
input_json_data = {"text": sample_text}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
ner_df = pd.DataFrame(data[0]["ner_predictions"])
assertion_df = pd.DataFrame(data[0]["assertion_predictions"])
relation_df = pd.DataFrame(data[0]["relation_predictions"])

In [10]:
ner_df

,ner_chunk,begin,end,ner_label,ner_confidence
0,man,14,16,Gender,0.9999
1,45 mm,119,123,Tumor_Size,0.84795
2,anterior,125,132,Direction,0.9987
3,mediastinal,134,144,Site_Other_Body_Part,0.8603
4,mass,146,149,Tumor_Finding,0.9344
5,chest CT,164,171,Imaging_Test,0.905
6,electromyography,203,218,Imaging_Test,0.9887
7,VGCC,317,320,Biomarker,0.9996
8,cisplatin,363,371,Chemotherapy,1.0
9,75 mg/m²,373,380,Dosage,0.82435


In [11]:
assertion_df

,ner_chunk,begin,end,ner_label,assertion,assertion_confidence
0,mass,146,149,Tumor_Finding,Present,0.9959
1,VGCC,317,320,Biomarker,Present,0.9999
2,cisplatin,363,371,Chemotherapy,Past,0.9001
3,etoposide,406,414,Chemotherapy,Present,0.793
4,for four cycles,462,476,Duration,Present,0.6795
5,video-assisted thoracic surgery,481,511,Cancer_Surgery,Past,1.0
6,small cell lung cancer,565,586,Carcinoma_Type,Present,0.9988
7,SCLC,589,592,Carcinoma_Type,Present,0.999
8,metastases,611,620,Metastasis,Present,1.0
9,AE1/AE3,684,690,Biomarker,Present,0.9999


In [12]:
relation_df

,ner_chunk1,ner_chunk1_begin,ner_chunk1_end,ner_label1,ner_chunk2,ner_chunk2_begin,ner_chunk2_end,ner_label2,relations,relation_confidence
0,cisplatin,363,371,Chemotherapy,75 mg/m²,373,380,Dosage,Chemotherapy-Dosage,1.0
1,cisplatin,363,371,Chemotherapy,day 1,385,389,Cycle_Day,Chemotherapy-Cycle_Day,1.0
2,cisplatin,363,371,Chemotherapy,100 mg/m²,416,424,Dosage,Chemotherapy-Dosage,1.0
3,cisplatin,363,371,Chemotherapy,days 1-3,429,436,Cycle_Day,Chemotherapy-Cycle_Day,1.0
4,75 mg/m²,373,380,Dosage,etoposide,406,414,Chemotherapy,Dosage-Chemotherapy,1.0
5,day 1,385,389,Cycle_Day,etoposide,406,414,Chemotherapy,Cycle_Day-Chemotherapy,1.0
6,etoposide,406,414,Chemotherapy,100 mg/m²,416,424,Dosage,Chemotherapy-Dosage,1.0
7,etoposide,406,414,Chemotherapy,days 1-3,429,436,Cycle_Day,Chemotherapy-Cycle_Day,1.0
8,45 mm,119,123,Tumor_Size,mass,146,149,Tumor_Finding,is_size_of,0.96841204
9,mediastinal,134,144,Site_Other_Body_Part,mass,146,149,Tumor_Finding,is_location_of,0.929259


### JSON Lines

In [13]:
import json

def create_jsonl(records):
    
    if isinstance(records, str):
        records = [records]
    
    json_records = []

    for text in records:
        record = {
            "text": text
        }
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record) for record in json_records)

    return json_lines

#### Example 1

In [14]:
input_jsonl_data = create_jsonl(sample_text)
data = process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"ner_predictions": [{"ner_chunk": "man", "begin": 14, "end": 16, "ner_label": "Gender", "ner_confidence": "0.9999"}, {"ner_chunk": "45 mm", "begin": 119, "end": 123, "ner_label": "Tumor_Size", "ner_confidence": "0.84795"}, {"ner_chunk": "anterior", "begin": 125, "end": 132, "ner_label": "Direction", "ner_confidence": "0.9987"}, {"ner_chunk": "mediastinal", "begin": 134, "end": 144, "ner_label": "Site_Other_Body_Part", "ner_confidence": "0.8603"}, {"ner_chunk": "mass", "begin": 146, "end": 149, "ner_label": "Tumor_Finding", "ner_confidence": "0.9344"}, {"ner_chunk": "chest CT", "begin": 164, "end": 171, "ner_label": "Imaging_Test", "ner_confidence": "0.905"}, {"ner_chunk": "electromyography", "begin": 203, "end": 218, "ner_label": "Imaging_Test", "ner_confidence": "0.9887"}, {"ner_chunk": "VGCC", "begin": 317, "end": 320, "ner_label": "Biomarker", "ner_confidence": "0.9996"}, {"ner_chunk": "cisplatin", "begin": 363, "end": 371, "ner_label": "Chemotherapy", "ner_confidence": "1.0"}, {"n

### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [15]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [16]:
import json
import os

input_dir = 'inputs/batch'
json_input_dir = f"{input_dir}/json"
jsonl_input_dir = f"{input_dir}/jsonl"

output_dir = 'outputs/batch'
json_output_dir = f"{output_dir}/json"
jsonl_output_dir = f"{output_dir}/jsonl"

os.makedirs(json_input_dir, exist_ok=True)
os.makedirs(jsonl_input_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(jsonl_output_dir, exist_ok=True)

validation_json_file_name = "input.json"

validation_jsonl_file_name = "input.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/jsonl/"

def write_and_upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    if file_format == ".json":
        input_data = json.dumps(input_data)

    with open(file_name, "w") as f:
        f.write(input_data)

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/batch/{file_format[1:]}/{os.path.basename(file_name)}",
        Body=(bytes(input_data.encode("UTF-8"))),
    )

In [17]:
input_jsonl_data = create_jsonl(docs)
input_json_data = {"text": docs}

write_and_upload_to_s3(input_json_data, f"{json_input_dir}/{validation_json_file_name}")

write_and_upload_to_s3(input_jsonl_data, f"{jsonl_input_dir}/{validation_jsonl_file_name}")

### JSON

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [19]:
from urllib.parse import urlparse

def process_s3_json_output_and_save(validation_file_name):

    output_file_path = f"{json_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    # Define a dictionary with DataFrame objects and their names
    display(data)
    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        json.dump(data, f_out, indent=4)

In [20]:
process_s3_json_output_and_save(validation_json_file_name)

[{'ner_predictions': [{'ner_chunk': 'She',
    'begin': 0,
    'end': 2,
    'ner_label': 'Gender',
    'ner_confidence': '1.0'},
   {'ner_chunk': 'computed tomography',
    'begin': 16,
    'end': 34,
    'ner_label': 'Imaging_Test',
    'ner_confidence': '0.93485'},
   {'ner_chunk': 'CT',
    'begin': 37,
    'end': 38,
    'ner_label': 'Imaging_Test',
    'ner_confidence': '0.9724'},
   {'ner_chunk': 'abdomen',
    'begin': 53,
    'end': 59,
    'ner_label': 'Site_Other_Body_Part',
    'ner_confidence': '0.9363'},
   {'ner_chunk': 'pelvis',
    'begin': 65,
    'end': 70,
    'ner_label': 'Site_Other_Body_Part',
    'ner_confidence': '0.5375'},
   {'ner_chunk': 'ovarian',
    'begin': 96,
    'end': 102,
    'ner_label': 'Site_Other_Body_Part',
    'ner_confidence': '0.7866'},
   {'ner_chunk': 'mass',
    'begin': 104,
    'end': 107,
    'ner_label': 'Tumor_Finding',
    'ner_confidence': '0.9573'},
   {'ner_chunk': 'Pap smear',
    'begin': 112,
    'end': 120,
    'ner_label': '

### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [22]:
from urllib.parse import urlparse

def process_s3_jsonlines_output_and_save(validation_file_name):

    output_file_path = f"{jsonl_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        for item in data.split('\n'):
            f_out.write(item + '\n')

In [23]:
process_s3_jsonlines_output_and_save(validation_jsonl_file_name)

{"ner_predictions": [{"ner_chunk": "She", "begin": 0, "end": 2, "ner_label": "Gender", "ner_confidence": "1.0"}, {"ner_chunk": "computed tomography", "begin": 16, "end": 34, "ner_label": "Imaging_Test", "ner_confidence": "0.93485"}, {"ner_chunk": "CT", "begin": 37, "end": 38, "ner_label": "Imaging_Test", "ner_confidence": "0.9724"}, {"ner_chunk": "abdomen", "begin": 53, "end": 59, "ner_label": "Site_Other_Body_Part", "ner_confidence": "0.9363"}, {"ner_chunk": "pelvis", "begin": 65, "end": 70, "ner_label": "Site_Other_Body_Part", "ner_confidence": "0.5375"}, {"ner_chunk": "ovarian", "begin": 96, "end": 102, "ner_label": "Site_Other_Body_Part", "ner_confidence": "0.7866"}, {"ner_chunk": "mass", "begin": 104, "end": 107, "ner_label": "Tumor_Finding", "ner_confidence": "0.9573"}, {"ner_chunk": "Pap smear", "begin": 112, "end": 120, "ner_label": "Pathology_Test", "ner_confidence": "0.96725"}, {"ner_chunk": "one month later", "begin": 132, "end": 146, "ner_label": "Relative_Date", "ner_confi

In [24]:
model.delete_model()

INFO:sagemaker:Deleting model with name: explain-clinical-doc-cancer-type-en-2024-09-20-07-50-49-623


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

